In [1]:
## Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import AutoDateFormatter, AutoDateLocator, WeekdayLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
import numpy as np
import datetime
%matplotlib gtk

In [2]:
## Import data frome pickle generated from hotline_daily.py
data = pd.read_pickle('pickle_hotline_0904.pkl')
data.head(3)

,month,year,0,1,2,3,4,5,6,7,...,21,22,23,24,summa,xlday,day,date,week,weekday
tix,11,2016,00:00-00:30,00:30-01:30,01:30-02:30,02:30-03:30,03:30-04:30,04:30-05:30,05:30-06:30,06:30-07:30,...,20:30-21:30,21:30-22:30,22:30-23:30,23:30-00:00,NaN,42688.0,14,2016-11-14,46,Mon
angekommen,11,2016,0,0,0,0,0,0,1,5,...,0,1,0,0,321,42688.0,14,2016-11-14,46,Mon
verbunden,11,2016,0,0,0,0,0,0,0,4,...,0,0,0,0,221,42688.0,14,2016-11-14,46,Mon


In [3]:
## rearrange dataframe
h_ix=range(0,25)
c_ix=['date','year','month','week','day','weekday','xlday','summa']
c_ix.extend(h_ix)
data=data[c_ix]
data.head(5)

,date,year,month,week,day,weekday,xlday,summa,0,1,...,15,16,17,18,19,20,21,22,23,24
tix,2016-11-14,2016,11,46,14,Mon,42688.0,NaN,00:00-00:30,00:30-01:30,...,14:30-15:30,15:30-16:30,16:30-17:30,17:30-18:30,18:30-19:30,19:30-20:30,20:30-21:30,21:30-22:30,22:30-23:30,23:30-00:00
angekommen,2016-11-14,2016,11,46,14,Mon,42688.0,321,0,0,...,18,14,9,7,5,0,0,1,0,0
verbunden,2016-11-14,2016,11,46,14,Mon,42688.0,221,0,0,...,18,13,8,5,5,0,0,0,0,0
verloren,2016-11-14,2016,11,46,14,Mon,42688.0,100,0,0,...,0,1,1,2,0,0,0,1,0,0
servicelevel,2016-11-14,2016,11,46,14,Mon,42688.0,0.688474,NaN,NaN,...,1,0.928571,0.888889,0.714286,1,NaN,NaN,0,NaN,NaN


**Dataframe kann hier noch nach Monaten, Wochen usw. gefiltert werden
momentan wird einfach der gesamte Datenbestand genommen
Alles, was später in den Plot kommt, kann gleich bleiben.
also: data = data[Filter hier; wochen etc.], danach kann alles normal weitergehen**

In [4]:
##split into dataframes with same index
angek_df=data.loc['angekommen'].set_index('xlday')
verbu_df=data.loc['verbunden'].set_index('xlday')
verlo_df=data.loc['verloren'].set_index('xlday')

## how many days?
datasetdays = angek_df.date.nunique()
print datasetdays

## have a glance
angek_df.head(3)

143


,date,year,month,week,day,weekday,summa,0,1,2,...,15,16,17,18,19,20,21,22,23,24
xlday,,,,,,,,,,,,,,,,,,,,,
42688.0,2016-11-14,2016,11,46,14,Mon,321,0,0,0,...,18,14,9,7,5,0,0,1,0,0
42689.0,2016-11-15,2016,11,46,15,Tue,152,0,0,0,...,13,17,0,5,4,1,0,0,0,0
42690.0,2016-11-16,2016,11,46,16,Wed,75,0,0,0,...,6,9,5,1,5,0,0,1,0,0


In [5]:
## this would be 12:30-13:30 or column 16 in the excelsheet
print verbu_df[13].sum(),verlo_df[13].sum(),angek_df[13].sum(),' this is correct'
print 'SLA für 12:30-13:30 auf alle Tage gerechnet wäre: ',verbu_df[13].sum()/angek_df[13].sum()

print verbu_df[9].sum(),verlo_df[9].sum(),angek_df[9].sum(),' this is correct'
print 'SLA für 8:30-9:30 auf alle Tage gerechnet wäre: ',verbu_df[9].sum()/angek_df[9].sum()

print verbu_df[h_ix].sum()



659.0 42.0 701.0  this is correct
SLA für 12:30-13:30 auf alle Tage gerechnet wäre:  0.940085592011
1033.0 210.0 1243.0  this is correct
SLA für 8:30-9:30 auf alle Tage gerechnet wäre:  0.83105390185
0        2.0
1        0.0
2        0.0
3        4.0
4        2.0
5        1.0
6       23.0
7      215.0
8      415.0
9     1033.0
10    1069.0
11     865.0
12     702.0
13     659.0
14     898.0
15     898.0
16     877.0
17     522.0
18     677.0
19     274.0
20      99.0
21      32.0
22      24.0
23       4.0
24       0.0
dtype: float64


In [6]:
## for the plot I'd like to have an X-Axis(=Index of df) of hours and a 
## Y-Axis(=columns) for verbunden,lost,angekommen plus the average sla per hour

hixframe = pd.DataFrame(index=h_ix)
hixframe['h_steps'] = data.loc['tix'][h_ix].iloc[0]
hixframe['angekommen'] = angek_df[h_ix].sum()
hixframe['verbunden'] = verbu_df[h_ix].sum()
hixframe['verloren'] = verlo_df[h_ix].sum()

hixframe['av_angekommen'] = angek_df[h_ix].sum()/datasetdays
hixframe['av_verbunden'] = verbu_df[h_ix].sum()/datasetdays
hixframe['av_verloren'] = verlo_df[h_ix].sum()/datasetdays

hixframe['sla_percent'] = (verbu_df[h_ix].sum()/angek_df[h_ix].sum())*100
hixframe.loc[[9,10,11]]

,h_steps,angekommen,verbunden,verloren,av_angekommen,av_verbunden,av_verloren,sla_percent
9,08:30-09:30,1243.0,1033.0,210.0,8.692308,7.223776,1.468531,83.105390
10,09:30-10:30,1253.0,1069.0,184.0,8.762238,7.475524,1.286713,85.315243
11,10:30-11:30,1005.0,865.0,140.0,7.027972,6.048951,0.979021,86.069652


In [7]:
## Create the plot from the summary dataframe
## X-Axis should be dates, Y-Axis should be number of calls

fig = plt.figure(figsize=(12,6))
fig.add_subplot(111)
plt.xlabel('Stunden')
plt.title('Durchschnittliche Stunde seit 14.11.')

# "Initialize" Axes to configure them later
ax = plt.axes()
ax.set_ylabel('Calls')
ax.set_axisbelow(True)

# Y-Axis1: major ticks every 50, minor ticks every 10                                    
minor_ticks_ax = np.arange(0, hixframe.av_angekommen.max(), 0.2)
major_ticks_ax = np.arange(0, hixframe.av_angekommen.max(), 1)
ax.yaxis.grid(True)
ax.set_yticks(minor_ticks_ax, minor=True)
ax.yaxis.grid(True, which='minor', color='#E8E6BF', linestyle='--', alpha=0.5, zorder=0)
ax.yaxis.grid(True, which='major', color='#6DC3EC', linestyle='-', zorder=3)

# X-Axis: locator set the ticks. No understanding of this, just copy-and-paste...
ax.set_xticks(hixframe.index[::2,]) # every second element from index, starting at 0
ax.set_xticks(hixframe.index[1::2,], minor=True) # every second element from index, starting at 1, set to minor ticks
ax.set_xticklabels(hixframe.h_steps[::2,])
ax.set_xticklabels(hixframe.h_steps[1::2,], minor=True)
ax.xaxis.set_tick_params(which='major', pad=1, length=20, width=1.5, labelsize=8)
ax.xaxis.set_tick_params(which='minor', pad=1, length=5, width=1.5, labelsize=8)

# Bars for each sum and day
ax.bar(hixframe.index,hixframe.av_angekommen,width=0.9, alpha=1, color='#8F8F8F', label='angekommen', align='center')
ax.bar(hixframe.index,hixframe.av_verbunden,width=0.2, alpha=1, color='#00B9FF', label='verbunden', align='center')
ax.bar(hixframe.index,hixframe.av_verloren,width=0.2, alpha=1, color='#FF0056', label='verloren', align='center')
ax.legend(loc=2)

# Markers for SLA
colors=['lightgreen' if i>80 else '#FFA2BE' for i in hixframe.sla_percent]
ax2 = ax.twinx() # for second scale "sla"
ax2.set_ylim([-100,105]) # Y-Axis 2: Set Label Range
minor_ticks_ax2 = np.arange(0, 105, 10)
ax2.set_yticks(minor_ticks_ax2)
ax2.axhline(y=80, color='#D7207A', alpha=0.5, linestyle='-', label="80% SLA") # show the desired SLA
ax2.set_ylabel('SLA %')
#ax2.plot(hixframe.index, hixframe.sla_percent, c='green', marker='8', markersize='4', linestyle='none', label='SLA %')
ax2.scatter(hixframe.index, hixframe.sla_percent, c=colors, label='SLA %')
ax2.legend(loc=1)


# Legend and Layout formatting after the plotting has taken place
plt.margins(0.01)

plt.tight_layout()